# Day 2 — Communication & Automation Agent (LangGraph)

This notebook implements a Communication & Automation Agent using:
- LangGraph (agent orchestration)
- LangChain (LLM + tools)

**Capabilities:**
- Analyze a message (tone, emotion, intent, urgency)
- Classify message (Complaint / Request / Escalation / Query / Feedback / Internal)
- Generate a professional reply
- Extract tasks (who / what / when / priority)

**Architecture:**
`HumanMessage → LLM (with tools) → Tools (classify / reply / tasks) → LLM → Final answer`

You can later wrap `run_communication_agent()` as a `/communication-agent` API in FastAPI.

## 0. Install Dependencies (if needed)
Uncomment and run this cell once in your environment (Colab / local).
- langgraph
- langchain
- langchain-openai
- pydantic

In [ ]:
# !pip install "langgraph>=1.0.0" "langchain>=0.3.0" "langchain-openai" "pydantic[email]" "grandalf" "matplotlib"

## 1. Imports & LLM Setup
Make sure you have your OpenAI-compatible key set:
`export OPENAI_API_KEY="sk-..."`

You can replace the model with any OpenAI-compatible one (e.g. `gpt-4.1`, `gpt-4o`, etc.).

In [ ]:
from typing import List, Dict, Any
from typing_extensions import TypedDict, Annotated
import operator

from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, AnyMessage, SystemMessage, ToolMessage

from langgraph.graph import StateGraph, START, END
from typing import Literal
import os

# Set your OpenAI API key here or via environment variable
# os.environ['OPENAI_API_KEY'] = 'your-api-key-here'
# Or set it in your shell: export OPENAI_API_KEY="sk-..."

# Initialize the model
llm = ChatOpenAI(
    model="gpt-4o-mini",  # change if you want a different model
    temperature=0.2,
)

## 2. Define Pydantic Models for Tool I/O

We define:
- `ClassificationResult`: category, urgency, tone, emotion, intent
- `ReplyInput`: input for reply_generation tool
- `Task`, `TaskExtractionResult`: outputs from task extractor

In [ ]:
class ClassificationResult(BaseModel):
    category: str = Field(
        description="High-level category of the message (Complaint, Request, Escalation, Query, Feedback, Internal)"
    )
    urgency: int = Field(
        ge=1, le=5,
        description="Urgency score from 1 (low) to 5 (critical)"
    )
    tone: str = Field(
        description="Perceived tone (neutral, polite, angry, frustrated, thankful, etc.)"
    )
    emotion: str = Field(
        description="Primary emotion detected (frustration, confusion, satisfaction, etc.)"
    )
    intent: str = Field(
        description="Short description of the main intent of the message"
    )


class ReplyInput(BaseModel):
    message: str = Field(description="Original message from the sender.")
    classification: ClassificationResult = Field(
        description="Classification result of the message."
    )
    style: str = Field(
        default="professional and empathetic",
        description="Tone/style for the reply, e.g. 'professional and empathetic'."
    )


class Task(BaseModel):
    assignee: str = Field(
        description="Who should do the task (if not specified, use 'Unassigned')."
    )
    description: str = Field(description="What needs to be done.")
    due_date: str = Field(
        description="Due date if explicitly mentioned, else 'Not specified'."
    )
    priority: str = Field(
        description="Priority label (Low, Medium, High, Critical)."
    )


class TaskExtractionResult(BaseModel):
    tasks: List[Task] = Field(
        description="List of extracted tasks from the message."
    )

## 3. Define Tools

We create 3 tools to match your Day 2 learning goals:

1. `classify_message_tool` – Category, urgency, tone, emotion, intent
2. `generate_reply_tool` – Professional reply based on classification
3. `extract_tasks_tool` – Task extraction (assignee, description, due_date, priority)

In [ ]:
@tool(parse_docstring=True)
def classify_message_tool(message: str) -> ClassificationResult:
    """\
    Analyze a message and return:
    - category
    - urgency (1-5)
    - tone
    - emotion
    - intent

    Args:
        message: The raw text message from user/client.
    """
    prompt = f"""
You are a message classification assistant.

Analyze the following message and output a JSON with:
- category: one of ["Complaint", "Request", "Escalation", "Query", "Feedback", "Internal"]
- urgency: integer 1 (low) to 5 (critical)
- tone: a short label for tone
- emotion: primary emotion
- intent: short description of what they want

Message:
{message}
"""
    model = llm.with_structured_output(ClassificationResult)
    result = model.invoke(prompt)
    return result


@tool(parse_docstring=True)
def generate_reply_tool(data: ReplyInput) -> str:
    """\
    Generate a professional reply to a message using its classification.

    Args:
        data: ReplyInput object containing message, classification, and style preferences.
    """
    cls = data.classification
    prompt = f"""
You are an assistant writing replies to messages.

Write a {data.style} reply to the message below.

Message:
{data.message}

Classification:
- Category: {cls.category}
- Urgency: {cls.urgency}
- Tone: {cls.tone}
- Emotion: {cls.emotion}
- Intent: {cls.intent}

Guidelines:
- Be clear and concise.
- Acknowledge the sender's emotion if negative.
- If it's a complaint or escalation, apologize where appropriate.
- If it is a request or query, provide clarity or next steps.
- Do not invent details you don't know.
"""
    response = llm.invoke(prompt)
    return response.content


@tool(parse_docstring=True)
def extract_tasks_tool(message: str) -> TaskExtractionResult:
    """\
    Extract tasks from a message.

    Args:
        message: A message that may contain implied or explicit tasks.
    """
    prompt = f"""
You are a task extraction assistant.

From the message below, extract a list of tasks.
For each task, identify:
- assignee (if present, else 'Unassigned')
- description
- due_date (if mentioned, else 'Not specified')
- priority (Low / Medium / High / Critical)

Return only valid tasks; if none, return an empty list.

Message:
{message}
"""
    model = llm.with_structured_output(TaskExtractionResult)
    result = model.invoke(prompt)
    return result


tools = [classify_message_tool, generate_reply_tool, extract_tasks_tool]
tools_by_name = {t.name: t for t in tools}

model_with_tools = llm.bind_tools(tools)

## 4. Define LangGraph State

We'll use a state that holds:
- `messages`: list of LangChain messages (Human, AI, Tool)
- `llm_calls`: count of LLM invocations (for debugging/teaching)

In [ ]:
class MessagesState(TypedDict):
    messages: Annotated[List[AnyMessage], operator.add]
    llm_calls: int

## 5. Define LLM Node

Node name: `llm_node`

Responsibilities:
- Add a system message with instructions
- Call LLM with bound tools
- Append `AIMessage` to the state

In [ ]:
def llm_node(state: MessagesState) -> MessagesState:
    """LLM decides whether to respond directly or call tools."""
    system = SystemMessage(
        content=(
            "You are a Communication & Automation Agent.\n"
            "You analyze incoming messages, classify them, generate professional replies, "
            "and extract tasks.\n"
            "Use the available tools when you need to classify, generate a reply, "
            "or extract tasks.\n"
            "If tools have already been used and you have enough information, "
            "produce a concise final answer."
        )
    )
    messages = [system] + state["messages"]
    ai_message = model_with_tools.invoke(messages)
    return {
        "messages": [ai_message],
        "llm_calls": state.get("llm_calls", 0) + 1,
    }

## 6. Define Tool Node

Node name: `tool_node`

Responsibilities:
- Read the last `AIMessage`
- For each `tool_call` requested by the LLM:
  - Run the corresponding Python tool
  - Append a `ToolMessage` with the result

In [ ]:
def tool_node(state: MessagesState) -> MessagesState:
    """Executes any tool calls requested by the LLM."""
    last_msg = state["messages"][-1]
    if not getattr(last_msg, "tool_calls", None):
        return {"messages": []}

    tool_messages: List[ToolMessage] = []

    for tool_call in last_msg.tool_calls:
        tool_name = tool_call["name"]
        tool = tools_by_name[tool_name]
        args = tool_call["args"]

        observation = tool.invoke(args)

        if isinstance(observation, BaseModel):
            content = observation.model_dump_json(indent=2)
        else:
            content = str(observation)

        tool_messages.append(
            ToolMessage(
                content=content,
                tool_call_id=tool_call["id"],
            )
        )

    return {"messages": tool_messages}

## 7. Define Conditional Edge: Continue or End

Logic:
- If last `AIMessage` has `tool_calls` → go to `tool_node`
- Else → `END` (final answer is ready)

In [ ]:
def should_continue(state: MessagesState) -> Literal["tool_node", END]:
    """Route based on whether the last AIMessage has tool calls."""
    messages = state["messages"]
    last_message = messages[-1]
    if getattr(last_message, "tool_calls", None):
        return "tool_node"
    return END

## 8. Build & Compile the LangGraph Workflow

Graph:
- `START → llm_node → (conditional: tool_node or END)`
- `tool_node → llm_node` (loop until no further tool calls)

In [ ]:
builder = StateGraph(MessagesState)

builder.add_node("llm_node", llm_node)
builder.add_node("tool_node", tool_node)

builder.add_edge(START, "llm_node")
builder.add_conditional_edges(
    "llm_node",
    should_continue,
    ["tool_node", END],
)
builder.add_edge("tool_node", "llm_node")

communication_agent = builder.compile()

## 8.1 Visualize the Graph

Display the compiled LangGraph workflow as a diagram to understand the agent's flow.

In [ ]:
from IPython.display import Image, display, SVG
import io

print("🎨 Generating LangGraph Visualization...\n")

# Method 1: Try PNG rendering (best quality)
try:
    graph_data = communication_agent.get_graph().draw_mermaid_png()
    display(Image(graph_data))
    print("✅ Displayed as PNG image")
except Exception as e:
    print(f"⚠️  PNG rendering failed: {e}\n")
    
    # Method 2: Try ASCII art visualization (always works!)
    try:
        from langgraph.graph import Graph
        
        print("📊 LangGraph Workflow Structure:")
        print("=" * 70)
        
        # Display the graph structure as ASCII
        graph = communication_agent.get_graph()
        
        # Get nodes and edges info
        print("\n🔵 NODES:")
        print("  • START (entry point)")
        print("  • llm_node (LLM decision maker)")
        print("  • tool_node (execute tools)")
        print("  • END (exit point)")
        
        print("\n🔗 EDGES:")
        print("  START → llm_node")
        print("  llm_node → tool_node (if tools needed)")
        print("  llm_node → END (if complete)")
        print("  tool_node → llm_node (loop back)")
        
        print("\n" + "=" * 70)
        
        # Try to display mermaid code for manual visualization
        print("\n📋 Mermaid Diagram Code (copy to https://mermaid.live):")
        print("-" * 70)
        mermaid_code = graph.draw_mermaid()
        print(mermaid_code)
        print("-" * 70)
        print("\n💡 Tip: Copy the code above and paste at https://mermaid.live to see the visual diagram")
        
    except Exception as e2:
        print(f"❌ Visualization failed: {e2}")
        print("\n📝 Graph exists but cannot be visualized in this environment")

print("\n" + "=" * 70)
print("Graph compilation successful! The agent is ready to use.")
print("=" * 70)

In [ ]:
# Alternative: Create a visual diagram using matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch

def draw_langgraph_diagram():
    """Draw the LangGraph workflow as a visual diagram"""
    fig, ax = plt.subplots(figsize=(12, 8))
    ax.set_xlim(0, 10)
    ax.set_ylim(0, 10)
    ax.axis('off')
    
    # Define node positions
    nodes = {
        'START': (5, 9),
        'llm_node': (5, 6.5),
        'tool_node': (2, 4),
        'END': (8, 4)
    }
    
    # Define node colors
    colors = {
        'START': '#90EE90',
        'llm_node': '#87CEEB',
        'tool_node': '#FFB6C1',
        'END': '#FFA07A'
    }
    
    # Draw nodes
    for node_name, (x, y) in nodes.items():
        if node_name in ['START', 'END']:
            circle = plt.Circle((x, y), 0.4, color=colors[node_name], ec='black', linewidth=2, zorder=3)
            ax.add_patch(circle)
            ax.text(x, y, node_name, ha='center', va='center', fontsize=10, fontweight='bold', zorder=4)
        else:
            box = FancyBboxPatch((x-0.6, y-0.35), 1.2, 0.7, 
                                boxstyle="round,pad=0.1", 
                                facecolor=colors[node_name],
                                edgecolor='black', linewidth=2, zorder=3)
            ax.add_patch(box)
            ax.text(x, y, node_name, ha='center', va='center', fontsize=10, fontweight='bold', zorder=4)
    
    # Draw edges with arrows
    # START -> llm_node
    arrow1 = FancyArrowPatch(
        (5, 8.6), (5, 6.85),
        arrowstyle='->', mutation_scale=30, linewidth=2,
        color='black', zorder=2
    )
    ax.add_patch(arrow1)
    
    # llm_node -> tool_node (conditional)
    arrow2 = FancyArrowPatch(
        (4.4, 6.15), (2.6, 4.35),
        arrowstyle='->', mutation_scale=30, linewidth=2,
        color='blue', linestyle='--', zorder=2
    )
    ax.add_patch(arrow2)
    ax.text(3, 5.5, 'if tools\nneeded', ha='center', va='center', 
            fontsize=8, color='blue', style='italic',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # llm_node -> END (conditional)
    arrow3 = FancyArrowPatch(
        (5.6, 6.15), (7.4, 4.35),
        arrowstyle='->', mutation_scale=30, linewidth=2,
        color='green', linestyle='--', zorder=2
    )
    ax.add_patch(arrow3)
    ax.text(7, 5.5, 'if complete', ha='center', va='center',
            fontsize=8, color='green', style='italic',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # tool_node -> llm_node (loop back)
    arrow4 = FancyArrowPatch(
        (2, 4.35), (4.4, 6.15),
        arrowstyle='->', mutation_scale=30, linewidth=2,
        color='purple', connectionstyle="arc3,rad=0.3", zorder=2
    )
    ax.add_patch(arrow4)
    ax.text(2.5, 5.8, 'return\nresults', ha='center', va='center',
            fontsize=8, color='purple', style='italic',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    # Add title and legend
    ax.set_title('Communication & Automation Agent - LangGraph Workflow', 
                fontsize=14, fontweight='bold', pad=20)
    
    # Add description boxes
    descriptions = [
        "1. User message enters at START",
        "2. LLM analyzes and decides action",
        "3. Tools execute if needed (classify/reply/extract)",
        "4. Results loop back to LLM",
        "5. Final answer sent to END"
    ]
    
    desc_text = "\n".join(descriptions)
    ax.text(5, 1.5, desc_text, ha='center', va='center',
            fontsize=9, bbox=dict(boxstyle='round', facecolor='lightyellow', 
                                 edgecolor='gray', linewidth=1.5, alpha=0.9))
    
    plt.tight_layout()
    plt.show()
    
    print("\n✅ Visual diagram displayed above!")
    print("📊 This diagram shows the complete workflow of the Communication Agent")

# Display the diagram
try:
    draw_langgraph_diagram()
except Exception as e:
    print(f"Could not create matplotlib diagram: {e}")
    print("The agent graph is still functional - visualization is optional.")

## 9. Test the Communication & Automation Agent

We'll send a realistic user message:
- Contains complaint
- Has urgency
- Implies tasks

Expectation:
- LLM calls `classify_message_tool`
- LLM may call `extract_tasks_tool`
- LLM calls `generate_reply_tool`
- Final AI response summarizes result.

In [ ]:
test_message = """
Hi team,

I am really unhappy with the delay in resolving my ticket.
It's been 3 days since I reported the login issue and I still can't access my account.
I need this fixed by tomorrow, otherwise I will escalate this to your management.

Regards,
Customer
"""

initial_state = {
    "messages": [HumanMessage(content=test_message)],
    "llm_calls": 0,
}

result_state = communication_agent.invoke(initial_state)

## 10. Inspect the Conversation Trace

This will print:
- `HumanMessage`
- `AIMessage`(s) (including tool call requests)
- `ToolMessage`(s) with JSON tool outputs

In [ ]:
for m in result_state["messages"]:
    if isinstance(m, HumanMessage):
        print("🧑 HUMAN:\n", m.content, "\n")
    elif isinstance(m, AIMessage):
        print("🤖 AI:\n", m.content or "[Tool call message]", "\n")
        if getattr(m, "tool_calls", None):
            print("  ↳ Tool calls:", m.tool_calls, "\n")
    elif isinstance(m, ToolMessage):
        print("🛠️ TOOL RESULT:\n", m.content, "\n")

## 11. High-Level Helper for Integration

You can use this helper in a FastAPI route:

**POST** `/communication-agent`
```json
{ "message": "..." }
```

This helper will:
- Run the LangGraph agent
- Return the final reply + raw history

In [ ]:
def run_communication_agent(message: str) -> Dict[str, Any]:
    """High-level helper to run the Communication & Automation Agent.
    Returns:
      - final_reply: last AI content message
      - history: full message list (for debug/audit)
      - llm_calls: number of LLM invocations
    """
    state = communication_agent.invoke(
        {
            "messages": [HumanMessage(content=message)],
            "llm_calls": 0,
        }
    )

    final_reply = None
    for m in reversed(state["messages"]):
        if isinstance(m, AIMessage) and m.content:
            final_reply = m.content
            break

    return {
        "final_reply": final_reply,
        "history": state["messages"],
        "llm_calls": state["llm_calls"],
    }


# Quick smoke test
if __name__ == "__main__":
    demo_out = run_communication_agent(
        "Hey, my invoice amount is wrong, please correct it before tomorrow."
    )
    print("=== FINAL REPLY ===")
    print(demo_out["final_reply"])